<a href="https://colab.research.google.com/github/omgits0mar/Datasets/blob/master/model1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jupyterthemes as jt
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme

ModuleNotFoundError: ignored

import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme

In [2]:
set_nb_theme('onedork')
#monokai
#chesterish
#oceans16 gamed
#onedork gamed brdo
#solarizedl

NameError: ignored

# Code start from here :

### Importing :

In [80]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import datetime

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from xgboost.sklearn import XGBRegressor


### Reading data from csv

In [81]:
df = pd.read_csv('airline-price-prediction.csv')

In [82]:
df.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price
0,5/3/2022,Vistara,UK,812,9:45,10h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...","56,588"
1,18-03-2022,Vistara,UK,975,5:45,06h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}","59,649"
2,9/3/2022,GO FIRST,G8,7537,14:30,08h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}","3,916"
3,15-03-2022,GO FIRST,G8,287,10:40,09h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}","6,031"
4,22-03-2022,Vistara,UK,826,12:30,07h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}","7,081"


In [83]:
df.shape

(240208, 11)

### Data preprocessing on 'price'

In [84]:
df['price'] = df['price'].str.replace(",", "")
df['price'] = pd.to_numeric(df['price'])

### Data preprocessing on 'date'

In [85]:
df["date"]=pd.to_datetime(df["date"])
df["date"]=df["date"].dt.strftime("%m/%d/%Y")


In [86]:
print(df[df.columns[0]])

0         05/03/2022
1         03/18/2022
2         09/03/2022
3         03/15/2022
4         03/22/2022
             ...    
240203    03/25/2022
240204    02/15/2022
240205    02/23/2022
240206    11/02/2022
240207    02/18/2022
Name: date, Length: 240208, dtype: object


In [87]:
pd.DatetimeIndex(df["date"]).weekday

Int64Index([1, 4, 5, 1, 1, 6, 0, 0, 0, 1,
            ...
            3, 3, 0, 2, 3, 4, 1, 2, 2, 4],
           dtype='int64', name='date', length=240208)

In [88]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df['dayofyear'] = pd.DatetimeIndex(df['date']).dayofyear
#df['weekofyear'] = pd.DatetimeIndex(df['date']).weekofyear

In [89]:
#df = pd.get_dummies(df, columns=['month'], drop_first=True, prefix='month')

### Data preprocessing : 'ch_code'

In [90]:
lbl_enc = LabelEncoder()
df['ch_code'] = lbl_enc.fit_transform(df[["ch_code"]])
print(df['ch_code'])

0         7
1         7
2         3
3         3
4         7
         ..
240203    2
240204    2
240205    1
240206    7
240207    6
Name: ch_code, Length: 240208, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Data preprocessing : 'dep_time'

In [91]:
#df["dep_time"]=pd.to_datetime(df["dep_time"])
#df['dep_time'] = df['dep_time'].dt.strftime("%-H:%M")

In [92]:
#df["dep_hour"]=pd.DatetimeIndex(df["dep_time"]).hour
#df["dep_minute"]=pd.DatetimeIndex(df["dep_time"]).minute

### Data preprocessing : 'time_taken'

In [93]:
df["hours_taken"] = df["time_taken"].str.split('h').str.get(0)
df["minutes_taken"] = df["time_taken"].str[4:6]
df["minutes_taken"] = df["minutes_taken"].str.replace('m', '')
df["minutes_taken"] = df["minutes_taken"].str.replace('h', '')
df["hours_taken"] = pd.to_numeric(df["hours_taken"])
df["minutes_taken"] = pd.to_numeric(df["minutes_taken"], errors='coerce')
df.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price,month,day,dayofyear,hours_taken,minutes_taken
0,05/03/2022,Vistara,7,812,9:45,10h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...",56588,5,3,123,10.0,10.0
1,03/18/2022,Vistara,7,975,5:45,06h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}",59649,3,18,77,6.0,30.0
2,09/03/2022,GO FIRST,3,7537,14:30,08h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}",3916,9,3,246,8.0,10.0
3,03/15/2022,GO FIRST,3,287,10:40,09h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}",6031,3,15,74,9.0,40.0
4,03/22/2022,Vistara,7,826,12:30,07h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}",7081,3,22,81,7.0,25.0


### Data preprocessing : 'stop'

In [94]:
df["stop"] = df["stop"].str.split('-').str.get(0)
df["stop"] = df["stop"].replace(['non'], 0)
df.isna().sum() #  28944 null vals
df["stop"] = df["stop"].replace(['2+'], 2) # Indicates for 2 or more stops
df['stop'] = df['stop'].fillna(0)
df['stop'] = pd.to_numeric(df['stop'])
# print(df[9:14])

### Data preprocessing : 'arr_time'

In [95]:
# df["arr_time"]=pd.to_datetime(df["arr_time"])
# df['arr_time'] = df['arr_time'].dt.strftime("%-H:%M")
# df["arr_hour"]=pd.DatetimeIndex(df["arr_time"]).hour
# df["arr_minute"]=pd.DatetimeIndex(df["arr_time"]).minute
# df["arr_hour"] = pd.to_numeric(df["arr_hour"])
# df["arr_minute"] = pd.to_numeric(df["arr_minute"])
# df.head()


### Data preprocessing : 'type'

In [96]:
df['type'] = lbl_enc.fit_transform(df[["type"]])
print(df['type'])

0         0
1         0
2         1
3         1
4         1
         ..
240203    1
240204    0
240205    1
240206    1
240207    1
Name: type, Length: 240208, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Data preprocessing : 'route'

In [97]:
df['source'] = df['route'].str.split( ', ').str.get(0).str.split(':').str.get(1)
df['destination'] = df['route'].str.split( ', ').str.get(1).str.split(':').str.get(1).str.split('}').str.get(0)
df['source'] = df['source'].str.replace('\'', "")
df['destination'] = df['destination'].str.replace('\'', "")

In [98]:
df.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price,month,day,dayofyear,hours_taken,minutes_taken,source,destination
0,05/03/2022,Vistara,7,812,9:45,10h 10m,1,19:55,0,"{'source': 'Bangalore', 'destination': 'Hydera...",56588,5,3,123,10.0,10.0,Bangalore,Hyderabad
1,03/18/2022,Vistara,7,975,5:45,06h 30m,1,12:15,0,"{'source': 'Delhi', 'destination': 'Hyderabad'}",59649,3,18,77,6.0,30.0,Delhi,Hyderabad
2,09/03/2022,GO FIRST,3,7537,14:30,08h 10m,1,22:40,1,"{'source': 'Mumbai', 'destination': 'Hyderabad'}",3916,9,3,246,8.0,10.0,Mumbai,Hyderabad
3,03/15/2022,GO FIRST,3,287,10:40,09h 40m,1,20:20,1,"{'source': 'Mumbai', 'destination': 'Delhi'}",6031,3,15,74,9.0,40.0,Mumbai,Delhi
4,03/22/2022,Vistara,7,826,12:30,07h 25m,1,19:55,1,"{'source': 'Chennai', 'destination': 'Kolkata'}",7081,3,22,81,7.0,25.0,Chennai,Kolkata


In [99]:
df['source'] = lbl_enc.fit_transform(df[["source"]])
print(df['source'])

0         0
1         2
2         5
3         5
4         1
         ..
240203    1
240204    2
240205    2
240206    2
240207    4
Name: source, Length: 240208, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [100]:
df['destination'] = lbl_enc.fit_transform(df[["destination"]])
print(df['destination'])
df = df.dropna()
df = df.drop(['airline', 'date', 'dep_time', "time_taken", 'arr_time', 'route',], axis=1)
df.head()
#df = pd.get_dummies(df)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0         3
1         3
2         3
3         2
4         4
         ..
240203    3
240204    4
240205    0
240206    5
240207    3
Name: destination, Length: 240208, dtype: int64


,ch_code,num_code,stop,type,price,month,day,dayofyear,hours_taken,minutes_taken,source,destination
0,7,812,1,0,56588,5,3,123,10.0,10.0,0,3
1,7,975,1,0,59649,3,18,77,6.0,30.0,2,3
2,3,7537,1,1,3916,9,3,246,8.0,10.0,5,3
3,3,287,1,1,6031,3,15,74,9.0,40.0,5,2
4,7,826,1,1,7081,3,22,81,7.0,25.0,1,4


In [101]:
print(df.columns)

Index(['ch_code', 'num_code', 'stop', 'type', 'price', 'month', 'day',
       'dayofyear', 'hours_taken', 'minutes_taken', 'source', 'destination'],
      dtype='object')


In [148]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df_scaled = pd.DataFrame(ss.fit_transform(df),columns = df.columns)
df_scaled.head()

,ch_code,num_code,stop,type,price,month,day,dayofyear,hours_taken,minutes_taken,source,destination
0,1.079057,-0.308075,0.191497,-1.489273,1.574051,0.450887,-1.416702,0.301149,-0.244881,-0.950153,-1.469249,0.236472
1,1.079057,-0.225785,0.191497,-1.489273,1.708911,-0.279549,0.184016,-0.291292,-0.800249,0.202175,-0.327885,0.236472
2,-0.512101,3.087034,0.191497,0.671468,-0.746538,1.911760,-1.416702,1.885283,-0.522565,-0.950153,1.384162,0.236472
3,-0.512101,-0.573121,0.191497,0.671468,-0.653357,-0.279549,-0.136128,-0.329929,-0.383723,0.778339,1.384162,-0.336562
4,1.079057,-0.301007,0.191497,0.671468,-0.607097,-0.279549,0.610874,-0.239775,-0.661407,-0.085907,-0.898567,0.809506


In [149]:
X = df_scaled.loc[:, df.columns != 'price']
Y = df_scaled['price']

In [150]:
dtree = DecisionTreeRegressor(max_depth=20, min_samples_leaf=0.10, random_state=8)

In [159]:
XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=120)

In [152]:
dtree.fit(XTrain, YTrain)

pred_train_tree= dtree.predict(XTrain)
print(np.sqrt(mean_squared_error(YTrain, pred_train_tree)))
print(r2_score(YTrain, pred_train_tree))

pred_test_tree= dtree.predict(XTest)
print(np.sqrt(mean_squared_error(YTest, pred_test_tree)))
print('Mean Square Error for testing set', metrics.mean_squared_error(YTest, pred_test_tree))
print(r2_score(YTest, pred_test_tree))

0.32362897568001125
0.8950689816271195
0.3239677533247199
Mean Square Error for testing set 0.10495510519426654
0.895812030929177


In [160]:
polynomial = PolynomialFeatures(degree=4)

XPolynomial = polynomial.fit_transform(XTrain)

Model = linear_model.LinearRegression()
Model.fit(XPolynomial, YTrain)

TrainPrediction = Model.predict(XPolynomial)

TesTPrediction = Model.predict(polynomial.fit_transform(XTest))

print(np.sqrt(mean_squared_error(YTest, TesTPrediction)))
print('Mean Square Error for testing set', metrics.mean_squared_error(YTest, TesTPrediction))
print(r2_score(YTest, TesTPrediction))

0.214107068000527
Mean Square Error for testing set 0.04584183656778229
0.954012145249301


# Drop arr_hour,min,dep_hour,min,weekofyear && Label encoder for all:
## No Scaling Applied :
### 26,699,895 -->Poly(d=2,r=120)
### 25,063,325 -->poly(d=3,r=120)------> Acc = 0.9512
### 48,657,835 -->Poly(d=5,r=120)------> Acc = 0.90525
### 25,498,894 -->Poly(d=4,r=120)------> Acc = 0.95035
### 24,642,093 -->Poly(d=4,r=8)------> Acc = 0.95251
### 44,714,500 -->ElasticNet(d=2,r=8)------> Acc = 0.91
## Applied Feature Scaling :
### 0.04603038 -->BEST Poly(d=4,r=8)------> Acc = 0.95430
### 0.10495510 -->Dtree
### 0.47465824 -->ElasticNet(d=2,r=8)------> Acc = 0.4746
 

In [154]:
import xgboost as xgb

data_dmatrix = xgb.DMatrix(data=X,label=Y)


xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.7, learning_rate = 0.001,
                max_depth = 20, alpha = 50, n_estimators = 20)
xg_reg.fit(XTrain,YTrain)

preds = xg_reg.predict(XTest)
print(np.sqrt(mean_squared_error(YTest, preds)))
print('Mean Square Error for testing set', metrics.mean_squared_error(YTest, preds))
print(r2_score(YTest, preds))

1.1005563010861696
Mean Square Error for testing set 1.2112241718604715
-0.20237111212505066


In [155]:
polynomial = PolynomialFeatures(degree=2)

XPolynomial = polynomial.fit_transform(XTrain)

Model = linear_model.ElasticNet()
Model.fit(XPolynomial, YTrain)

TrainPrediction = Model.predict(XPolynomial)

TesTPrediction = Model.predict(polynomial.fit_transform(XTest))

print(np.sqrt(mean_squared_error(YTest, TesTPrediction)))
print('Mean Square Error for testing set', metrics.mean_squared_error(YTest, TesTPrediction))
print(r2_score(YTest, TesTPrediction))

0.7274681039298428
Mean Square Error for testing set 0.5292098422352807
0.4746582496132209


In [107]:
df.head()

,ch_code,num_code,stop,type,price,month,day,dayofyear,hours_taken,minutes_taken,source,destination
0,7,812,1,0,56588,5,3,123,10.0,10.0,0,3
1,7,975,1,0,59649,3,18,77,6.0,30.0,2,3
2,3,7537,1,1,3916,9,3,246,8.0,10.0,5,3
3,3,287,1,1,6031,3,15,74,9.0,40.0,5,2
4,7,826,1,1,7081,3,22,81,7.0,25.0,1,4
